<a href="https://colab.research.google.com/github/mukaseevru/skillfactory_rds/blob/master/final/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Поддержка Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports
Импортируем необходимые библиотеки

In [ ]:
import os
import re
import csv
import numpy as np
import pandas as pd
# import codecs
from string import punctuation
from collections import defaultdict
# from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, LSTM, Lambda
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalAveragePooling1D
import keras.backend as K

# Settings
Ниже установим необходимые настройки модели

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/skillfactory/Project-Final/'
PATH_DATA = PATH + 'data/'
embedding_file = PATH_DATA + 'glove.840B.300d.txt'
train_data_file = PATH_DATA + 'train.csv'
test_data_file = PATH_DATA + 'test.csv'
max_sequence_length = 60
max_num_words = 200_000 # Всего около 201000 уникальных слов в обучающем датасете. 200000 слов достаточно для токенизации
embedding_dim = 300
validation_split_ratio = 0.1
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
lstm_name = 'lstm_{:d}_{:d}_{:.2f}_{:.2f}'.format(num_lstm, num_dense, rate_drop_lstm, rate_drop_dense)
act_f = 'relu'
re_weight = True
np.random.seed(21)

print(lstm_name)

lstm_231_144_0.33_0.38


# Functions
Определим необходимые функции

In [ ]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    '''
    Функция очистки текста. В качестве опции удаление стопслов и
    привидение слов в начальную форму
    '''
    text = text.lower().split()
    # Опционально, удаление стоп слов
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        text = [w for w in text if not w in stop_words]
    text = " ".join(text)
    # Удаление пунктуации
    text = "".join([c for c in text if c not in punctuation])
    # Очистка текста
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Опционально, привидение слов к начальном форме
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    return(text)


def q1_q2_intersect(row):
    '''
    Функция расчета пересечений
    '''
    return(len(set(q_dict[row['question1']]).intersection(set(q_dict[row['question2']]))))


def q1_freq(row):
    '''
    Функция расчета частотности question1
    '''
    return(len(q_dict[row['question1']]))


def q2_freq(row):
    '''
    Функция расчета частотности question2
    '''
    return(len(q_dict[row['question2']]))

# Create word embedding dictionary
Для создания используется файл 'glove.840B.300d.txt'

In [ ]:
embeddings_index = {}
f = open(embedding_file, encoding='utf-8')

# for line in tqdm(f):
for line in f:
    values = line.split()
    # word = values[0]
    word = ''.join(values[:-300])   
    # coefs = np.asarray(values[1:], dtype='float32')
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print(f'Found {len(embeddings_index)} word vectors.')

Found 2195892 word vectors.


# Process text in dataset
Обработка текста в датасете

In [ ]:
train_texts_1 = [] 
train_texts_2 = []
train_labels = []

df_train = pd.read_csv(train_data_file, encoding='utf-8')
df_train = df_train.fillna('empty')
train_q1 = df_train['question1'].values
train_q2 = df_train['question2'].values
train_labels = df_train['is_duplicate'].values

In [ ]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404290 non-null  object
 4   question2     404290 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [ ]:
for text in train_q1:
    train_texts_1.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))
    
for text in train_q2:
    train_texts_2.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))

print(f'{len(train_texts_1)} texts are found in train.csv')

404290 texts are found in train.csv


In [ ]:
test_texts_1 = []
test_texts_2 = []
test_ids = []

df_test = pd.read_csv(test_data_file, encoding='utf-8', low_memory=False)
df_test = df_test.fillna('empty')
test_q1 = df_test['question1'].values
test_q2 = df_test['question2'].values
test_ids = df_test['test_id'].values

In [ ]:
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   test_id    int64 
 1   question1  object
 2   question2  object
dtypes: int64(1), object(2)
memory usage: 53.7+ MB


In [ ]:
for text in test_q1:
    test_texts_1.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))
    
for text in test_q2:
    test_texts_2.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))

print(f'{len(test_texts_1)} texts are found in test.csv')

2345796 texts are found in test.csv


# Tokenize words in all sentences
Проведем токенезацию текста

In [ ]:
tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_texts_1 + train_texts_2 + test_texts_1 + test_texts_2)

train_sequences_1 = tokenizer.texts_to_sequences(train_texts_1)
train_sequences_2 = tokenizer.texts_to_sequences(train_texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print(f'{len(word_index)} unique tokens are found')

159606 unique tokens are found


In [ ]:
# pad all train with Max_Sequence_Length
train_data_1 = pad_sequences(train_sequences_1, maxlen=max_sequence_length)
train_data_2 = pad_sequences(train_sequences_2, maxlen=max_sequence_length)
print(f'Shape of train data tensor: {train_data_1.shape}')
print(f'Shape of train labels tensor: {train_labels.shape}')

# pad all test with Max_Sequence_Length
test_data_1 = pad_sequences(test_sequences_1, maxlen=max_sequence_length)
test_data_2 = pad_sequences(test_sequences_2, maxlen=max_sequence_length)
print(f'Shape of test data tensor: {test_data_2.shape}')
print(f'Shape of test ids tensor:{test_ids.shape}')

Shape of train data tensor: (404290, 60)
Shape of train labels tensor: (404290,)
Shape of test data tensor: (2345796, 60)
Shape of test ids tensor:(2345796,)


# Leaky features

In [ ]:
questions = pd.concat([df_train[['question1', 'question2']], \
                       df_test[['question1', 'question2']]], axis=0).reset_index(drop='index')
questions.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [ ]:
q_dict = defaultdict(set)
for i in range(questions.shape[0]):
    q_dict[questions.question1[i]].add(questions.question2[i])
    q_dict[questions.question2[i]].add(questions.question1[i])

In [ ]:
print(questions.question1[0])
q_dict[questions.question1[0]]

What is the step by step guide to invest in share market in india?


{'What is the step by step guide to invest in share market?'}

In [ ]:
print(questions.question2[0])
q_dict[questions.question2[0]]

What is the step by step guide to invest in share market?


{'What is my literal step by step guide (which website, how to open accounts, etc) on how to invest in index-managed ETFs? I live in Canada.',
 'What is the step by step guide to invest in share market in india?'}

In [ ]:
df_train['q1_q2_intersect'] = df_train.apply(q1_q2_intersect, axis=1)
df_train['q1_freq'] = df_train.apply(q1_freq, axis=1)
df_train['q2_freq'] = df_train.apply(q2_freq, axis=1)

df_test['q1_q2_intersect'] = df_test.apply(q1_q2_intersect, axis=1)
df_test['q1_freq'] = df_test.apply(q1_freq, axis=1)
df_test['q2_freq'] = df_test.apply(q2_freq, axis=1)

In [ ]:
train_leaks = df_train[['q1_q2_intersect', 'q1_freq', 'q2_freq']]
test_leaks = df_test[['q1_q2_intersect', 'q1_freq', 'q2_freq']]
train_leaks.head()

,q1_q2_intersect,q1_freq,q2_freq
0,0,1,2
1,0,8,3
2,0,2,1
3,0,1,1
4,0,3,1


In [ ]:
ss = StandardScaler()
ss.fit(np.vstack((train_leaks, test_leaks)))
train_leaks = ss.transform(train_leaks)
test_leaks = ss.transform(test_leaks)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Create embedding matrix for embedding layer
Создадим матрицу эмбеддингов по нашим словам

In [ ]:
num_words = min(max_num_words, len(word_index))+1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(f'Null word embeddings: {np.sum(np.sum(embedding_matrix, axis=1) == 0)}')

Null word embeddings: 68603


# Train Validation split

In [ ]:
perm = np.random.permutation(len(train_data_1))
idx_train = perm[:int(len(train_data_1)*(1-validation_split_ratio))]
idx_val = perm[int(len(train_data_1)*(1-validation_split_ratio)):]

In [ ]:
data_1_train = np.vstack((train_data_1[idx_train], train_data_2[idx_train]))
data_2_train = np.vstack((train_data_2[idx_train], train_data_1[idx_train]))
leaks_train = np.vstack((train_leaks[idx_train], train_leaks[idx_train]))
labels_train = np.concatenate((train_labels[idx_train], train_labels[idx_train]))

data_1_val = np.vstack((train_data_1[idx_val], train_data_2[idx_val]))
data_2_val = np.vstack((train_data_2[idx_val], train_data_1[idx_val]))
leaks_val = np.vstack((train_leaks[idx_val], train_leaks[idx_val]))
labels_val = np.concatenate((train_labels[idx_val], train_labels[idx_val]))

In [ ]:
weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.471544715
    weight_val[labels_val==0] = 1.309033281

In [ ]:
weight_val

array([1.30903328, 1.30903328, 1.30903328, ..., 0.47154472, 0.47154472,
       1.30903328])

# The embedding layer containing the word vectors
Определим архитектуру нейронной сети

In [ ]:
emb_layer = Embedding(
    input_dim=num_words,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_sequence_length,
    trainable=False
)    

# LSTM layer
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

# Define inputs
seq1 = Input(shape=(max_sequence_length,), dtype='int32')
seq2 = Input(shape=(max_sequence_length,), dtype='int32')

# Run inputs through embedding
emb1 = emb_layer(seq1)
emb2 = emb_layer(seq2)

# Run through LSTM layers
lstm_a = lstm_layer(emb1)
# glob1a = GlobalAveragePooling1D()(lstm_a)
lstm_b = lstm_layer(emb2)
# glob1b = GlobalAveragePooling1D()(lstm_b)

magic_input = Input(shape=(train_leaks.shape[1],))
# magic_dense = BatchNormalization()(magic_input)
magic_dense = Dense(int(num_dense/2), activation=act_f)(magic_input)

merged = concatenate([lstm_a, lstm_b, magic_dense])
merged = BatchNormalization()(merged)
merged = Dropout(rate_drop_dense)(merged)

merged = Dense(num_dense, activation=act_f)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(rate_drop_dense)(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [ ]:
# Add class weight
if re_weight:
    class_weight = {0: 1.309033281, 1: 0.471544715}
else:
    class_weight = None

# Train the model
Проведем обучение модели

In [ ]:
model = Model(inputs=[seq1, seq2, magic_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

In [ ]:
# Set early stopping (large patience should be useful)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
bst_model_path = PATH + 'result/' + lstm_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [ ]:
hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
                 validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
                 epochs=200, batch_size=2048, shuffle=True, class_weight=class_weight, \
                 callbacks=[early_stopping, model_checkpoint])

Epoch 1/200
356/356 [==============================] - 187s 501ms/step - loss: 0.2889 - acc: 0.8203 - val_loss: 0.2891 - val_acc: 0.8011
Epoch 2/200
356/356 [==============================] - 171s 480ms/step - loss: 0.2451 - acc: 0.8342 - val_loss: 0.2327 - val_acc: 0.8368
Epoch 3/200
356/356 [==============================] - 169s 474ms/step - loss: 0.2339 - acc: 0.8377 - val_loss: 0.2212 - val_acc: 0.8427
Epoch 4/200
356/356 [==============================] - 169s 473ms/step - loss: 0.2252 - acc: 0.8415 - val_loss: 0.2167 - val_acc: 0.8473
Epoch 5/200
356/356 [==============================] - 169s 474ms/step - loss: 0.2182 - acc: 0.8448 - val_loss: 0.2140 - val_acc: 0.8477
Epoch 6/200
356/356 [==============================] - 170s 478ms/step - loss: 0.2129 - acc: 0.8478 - val_loss: 0.2054 - val_acc: 0.8449
Epoch 7/200
356/356 [==============================] - 167s 470ms/step - loss: 0.2079 - acc: 0.8504 - val_loss: 0.2063 - val_acc: 0.8630
Epoch 8/200
356/356 [====================

In [ ]:
model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

# Make the submission
Сделаем файл с предсказаниями для kaggle

In [ ]:
preds = model.predict([test_data_1, test_data_2, test_leaks], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1, test_leaks], batch_size=8192, verbose=1)
preds /= 2

287/287 [==============================] - 132s 461ms/step


In [ ]:
submission = pd.DataFrame({'test_id': test_ids,
                           'is_duplicate': preds.ravel()})
submission.to_csv(PATH + f'result/{bst_val_score}_{lstm_name}_with_glove_embedding.csv', index=False)

# Conclusion

Заключение
1. Удалось создать модель, которая предсказывает вероятность дубликата вопроса.
2. Ошибка модели в соревновании на kaggle составила: 0.18611 (680 место в Private из 3295, ТОП-20%).
3. Модель планируется использовать для предсказания вероятности повтора закупок, используя информацию из назначения платежей.
4. Для использования модели реализован сервис, написанный на Flask: [server.py](https://github.com/mukaseevru/skillfactory_rds/blob/master/final/server.py) Запускать можно в докер контейнере.